# A (very) simple waveform generator using Plotly's streaming API
####You'll probably want to use something else than ossaudiodev...
#### If you want support for other format that wav, using pydub should be fairly straightforward

In [1]:
N = 4 # Number of intervals per sec (20 max ?)
FILENAME = 'bonobo_pieces.wav' # Obviously choose your own song

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline

In [3]:
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
# py.iplot(data, filename = 'graph_name', fileopt='mode')

#### Use your own stream tokens

In [4]:
tls.set_credentials_file(stream_ids=[
        'stream_token1',
        'stream_token_2',
        'stream_token_3',
        'stream_token_4'])
stream_ids = tls.get_credentials_file()['stream_ids']

stream_id0 = stream_ids[0]
stream0 = Stream(
    token=stream_id0,
    #maxpoints=5000
)

stream_id1 = stream_ids[1]
stream1 = Stream(
    token=stream_id1,
    #maxpoints=5000
)

In [5]:
trace1 = Scatter(
    name='Channel 1',
    x=[],
    y=[],
    stream=stream0,

    mode='lines+markers',
    line=Line(
        shape='spline'
    )
)

trace2 = Scatter(
    name='Channel 2',
    x=[],
    y=[],
    stream=stream1, 
    
    mode='lines+markers',
    line=Line(
        shape='spline'
    )
)

data = Data([trace1, trace2])

In [6]:
layout = Layout(title='Waveform')

fig = Figure(data=data, layout=layout)

unique_url = py.plot(fig, filename='waveform-first-stream')

In [7]:
s0 = py.Stream(stream_id0)
s1 = py.Stream(stream_id1)
s0.open()
s1.open()

#Wav file format


A wav files' data stream usually goes like this:

[ch1][ch2][ch1][ch2]...

where each [ch1][ch2] pair is a frame (asssuming 2 channels of course)

In [8]:
import wave
audio_file = wave.open(FILENAME)

In [9]:
nchannels, sampwidth, framerate, nframes, comptype, compname = audio_file.getparams()
data = np.fromstring(audio_file.readframes(-1), np.int16)

# Split channels (Currently this nb works only for 2 channels files since streams are "hardcoded")
channels = []
for channel in range(nchannels):
    channels.append(data[channel::nchannels])

In [10]:
# Blocking...Might need to kill processes that outputs sound?
from ossaudiodev import open as ossOpen
dsp = ossOpen('/dev/dsp', 'w')

try:
    from ossaudiodev import AFMT_S16_NE
except ImportError:
    if byteorder == 'little':
        AFMT_S16_NE = ossaudiodev.AFMT_S16_LE
    else:
        AFMT_S16_NE = ossaudiodev.AFMT_S16_BE
        
dsp.setparameters(AFMT_S16_NE, nchannels, framerate)

(16, 2, 44100)

In [11]:
# Song length (in seconds)
# This will be the "streaming" rate
length = len(data) // nchannels // framerate
intervals = length * N # Plotly can plot up to 20 pts per sec?

In [ ]:
for x in range(intervals):
    _data = data[:framerate*nchannels/N]
    dsp.write(_data)
    #s0.write(dict(x=x, y=np.mean(channels[0][:framerate/N])))
    #s1.write(dict(x=x, y=np.mean(channels[1][:framerate/N])))
    s0.write(dict(x=x, y=channels[0][x]))
    s1.write(dict(x=x, y=channels[1][x]))
    
    # Update
    data = data[framerate*nchannels/N:]
    channels[0] = channels[0][framerate/N:]
    channels[1] = channels[1][framerate/N:]

In [28]:
# Reset
audio_file.rewind()
data = np.fromstring(audio_file.readframes(-1), np.int16)
channels = []
for channel in range(nchannels):
    channels.append(data[channel::nchannels])
    
unique_url = py.plot(fig, filename='waveform-first-stream')

In [ ]:
# Close
audio_file.close()
dsp.close()